## Importing the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import re
import json

Creating Review class to organize the Reviews.

In [2]:
class Review:
    
    def __init__(self, text, rating):
        self.text = text
        self.rating = rating
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.rating > 3:
            return 'Positive'
        elif self.rating < 3:
            return 'Negative'
        else:
            return 'Neutral'

In [3]:
# Dataset path
file_name = 'D:/Git_Projects/SentimentAnalysis/Datasets/Books_small_10000.json'

In [4]:
#Fetching and storing reviews
reviews = []

with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

In [5]:
# Converting data into a Dataframe to preprocess it in an optimal way.
df_reviews = pd.DataFrame({'Review': [reviews[i].text for i in range(len(reviews))],
                            'Ratings': [reviews[i].rating for i in range(len(reviews))],
                            'Sentiment': [reviews[i].sentiment for i in range(len(reviews))]})

### Balancing Dataset

Balancing the number of reviews w.r.t sentiment.

In [6]:
import random
def balance_reviews(reviews):
    
    balance_value = df_reviews['Sentiment'].value_counts()['Negative']
    positive_counter = 0
    negative_counter = 0
    balanced_reviews = []
    
    for i in range(len(df_reviews)):
        if (df_reviews['Sentiment'][i] == 'Positive') and (positive_counter < balance_value):
            balanced_reviews.append(df_reviews.iloc[i, :])
            positive_counter += 1
            
        elif (df_reviews['Sentiment'][i] == 'Negative') and (negative_counter < balance_value):
            balanced_reviews.append(df_reviews.iloc[i, :])
            negative_counter += 1
        
        else:
            pass
    
    return pd.DataFrame(balanced_reviews)

In [7]:
balanced_df = balance_reviews(df_reviews)

In [8]:
balanced_df.head()

,Review,Ratings,Sentiment
0,"I bought both boxed sets, books 1-5. Really a...",5.0,Positive
2,I love Nicholas Sparks. I&#8217;ve read everyt...,4.0,Positive
3,I really enjoyed this adventure and look forwa...,4.0,Positive
5,I hoped for Mia to have some peace in this boo...,5.0,Positive
6,The book has the fevered intensity of Oliver S...,2.0,Negative


## Pre-Processing the Reviews
### Subtasks:
1) Converting alphabets into lowercase.
2) Tokenizing the review text, i.e. converting the words present in the review into a list.
3) Removing Digits from the Review.
3) Removing Stopwords from the review, where these stopwords do not add value to predicting the nature of a review, like I, me.
4) Converting the words in past, present, and future, continuous and various tense to one single tense.

Step No. 3, 4, & 5 reduces the size of our metrics (where column_names of the metrics contains a single word, rows contains the review number and the data field contains the number of times a particular word is present inside a review. )

In [9]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
def preprocess_review(review):
    '''
    Input:
    reviews: a list of reviews, where every review is in a textual form.
    
    Output:
    review_list: Reviews after going through preprocessing steps.
    '''
    pp_review = ''
    non_alphanumeric = re.compile('[\W]+')
    lancaster = LancasterStemmer()
    
    lower = review.lower()
    clean_review = re.compile('[\d]+').sub('', non_alphanumeric.sub(' ', lower)) # Removing Digits from the review.
    words = word_tokenize(clean_review)
    stemmed_words = []

    for word in words:
        if word not in stopwords.words('english'):
            stemmed_words.append(lancaster.stem(word))
        else:
            pass

    pp_review = ' '.join(stemmed_words)
        
    return pp_review

In [10]:
# Creating a column named Preprocessed_Reviews which contains the reviews preprocessed using the preprocessed_review function.
balanced_df['Preprocessed_Review'] = balanced_df['Review'].apply(preprocess_review)

In [11]:
#balanced_df.head()

In [12]:
final_df = pd.get_dummies(balanced_df, columns=['Sentiment'], drop_first=True)
final_df.head()

,Review,Ratings,Preprocessed_Review,Sentiment_Positive
0,"I bought both boxed sets, books 1-5. Really a...",5.0,bought box set book real gre sery start book t...,1
2,I love Nicholas Sparks. I&#8217;ve read everyt...,4.0,lov nichola spark read everyth writ wait cop s...,1
3,I really enjoyed this adventure and look forwa...,4.0,real enjoy adv look forward read robert spir e...,1
5,I hoped for Mia to have some peace in this boo...,5.0,hop mia peac book story real raw brok world to...,1
6,The book has the fevered intensity of Oliver S...,2.0,book fev intens ol ston movy jfk auth track de...,0


### CountVectorize

CountVector counts the number of words present in text. (Which word, in the text, is present and how many time)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

countvector = CountVectorizer()
count_transform = countvector.fit(final_df['Preprocessed_Review'])
X = count_transform.transform(final_df['Preprocessed_Review'])

In [14]:
sparse_df = pd.DataFrame(X.toarray(), columns=countvector.get_feature_names())

## Machine Learning

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(sparse_df, final_df['Sentiment_Positive'], test_size = 0.1, random_state=0)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train).reshape(-1, 1)
Y_test = np.array(Y_test).reshape(-1, 1)

## LogisticRegression

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [17]:
logreg = LogisticRegression(C=1)
logreg.fit(X_train, Y_train)

C:\Users\Mann\anaconda3\envs\py3tf2\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
f1_score(logreg.predict(X_test), Y_test)

0.823529411764706

## Training the data inside a Deep Neural Network.

In [19]:
from tensorflow import keras

In [20]:
model = keras.Sequential([
    keras.layers.Dense(1024, activation='tanh'),
    #keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2, activation='sigmoid')
])

In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(x=X_train, y=Y_train, batch_size=16, epochs=5, validation_data=(X_test, Y_test))

Train on 1159 samples, validate on 129 samples
Epoch 1/5
1159/1159 [==============================] - 4s 3ms/sample - loss: 0.5287 - accuracy: 0.7489 - val_loss: 0.4411 - val_accuracy: 0.8605
Epoch 2/5
1159/1159 [==============================] - 3s 3ms/sample - loss: 0.1557 - accuracy: 0.9456 - val_loss: 0.4274 - val_accuracy: 0.8682
Epoch 3/5
1159/1159 [==============================] - 3s 3ms/sample - loss: 0.0588 - accuracy: 0.9802 - val_loss: 0.6459 - val_accuracy: 0.8682
Epoch 4/5
1159/1159 [==============================] - 3s 3ms/sample - loss: 0.0231 - accuracy: 0.9940 - val_loss: 0.7142 - val_accuracy: 0.8295
Epoch 5/5
1159/1159 [==============================] - 3s 3ms/sample - loss: 0.0121 - accuracy: 0.9983 - val_loss: 0.7206 - val_accuracy: 0.8527


In [23]:
def get_sentiment(review):
    pp_review = preprocess_review(review)
    transformed = count_transform.transform([pp_review])
    array = transformed.toarray()
    return model.predict_classes(array.reshape(1, -1))

In [24]:
ex1 = 'This is a nice Book!'

In [25]:
get_sentiment(ex1)

array([1], dtype=int64)